<a href="https://colab.research.google.com/github/NicolaCortinovis/MLOPS_Project/blob/main/research/finetuning_transoformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FINETUNE THE FLAN-T5-SMALL MODEL
--------------------------------------------------------------------------------
Hugging Face page of the model --> [link](https://huggingface.co/google/flan-t5-small)
--------------------------------------------------------------------------------

In this Colab Notebook we finetune the Flan-T5-small to perform QAG task.

## PREPARE THE DATASET

In [ ]:
! pip install transformers datasets
! pip install transformers[torch]
! pip install --upgrade tensorflow

In [ ]:
from datasets import load_dataset
dataset = load_dataset("lmqg/qag_squad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset["train"]

Dataset({
    features: ['answers', 'questions', 'paragraph', 'questions_answers'],
    num_rows: 16462
})

In [ ]:
dataset['validation']

Dataset({
    features: ['answers', 'questions', 'paragraph', 'questions_answers'],
    num_rows: 2067
})

In [ ]:
dataset['test']

Dataset({
    features: ['answers', 'questions', 'paragraph', 'questions_answers'],
    num_rows: 2429
})

In [ ]:
dataset['train'][0]

{'answers': ['4 Minutes',
  'Elvis Presley',
  'thirteenth',
  'Sticky & Sweet Tour',
  '$280 million,'],
 'questions': ["Which single was released as the album's lead single?",
  'Madonna surpassed which artist with the most top-ten hits?',
  "4 minutes became Madonna's which number one single in the UK?",
  'What is the name of the first tour with Live Nation?',
  'How much did Stick and Sweet Tour grossed?'],
 'paragraph': '"4 Minutes" was released as the album\'s lead single and peaked at number three on the Billboard Hot 100. It was Madonna\'s 37th top-ten hit on the chart—it pushed Madonna past Elvis Presley as the artist with the most top-ten hits. In the UK she retained her record for the most number-one singles for a female artist; "4 Minutes" becoming her thirteenth. At the 23rd Japan Gold Disc Awards, Madonna received her fifth Artist of the Year trophy from Recording Industry Association of Japan, the most for any artist. To further promote the album, Madonna embarked on th

Now we need a tokenizer to process the text and include a padding and truncation strategy tho handle any variable sequence lenths.

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

Check this [link](https://www.philschmid.de/fine-tune-flan-t5#2-load-and-prepare-samsum-dataset) for more info about this preprocessing

In [ ]:
from datasets import concatenate_datasets

In [ ]:
tokenized_inputs = concatenate_datasets([dataset["train"],dataset["validation"],dataset["test"]]).map(lambda x: tokenizer(x["paragraph"], truncation=True), batched=True, remove_columns=['answers', 'questions', 'paragraph', 'questions_answers'])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

tokenized_targets = concatenate_datasets([dataset["train"],dataset["validation"], dataset["test"]]).map(lambda x: tokenizer(x["questions_answers"], truncation=True), batched=True, remove_columns=['answers', 'questions', 'paragraph', 'questions_answers'])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Max source length: 512
Max target length: 512


In [ ]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["Generate question and answer: " + item for item in sample["paragraph"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["questions_answers"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs




In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["paragraph", "questions_answers", "answers","questions"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/2067 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


The evaluation on the validation set will consume much ram. To speed up the process in this notebook and avoid to go out of ram, we select a small subset for the validation set:

In [ ]:

small_eval_dataset = tokenized_dataset["validation"].shuffle(seed=42).select(range(10))

## FINETUNING AND EVALUATION



Let's import the pretrained model

In [ ]:
from transformers import AutoModelForSeq2SeqLM

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

In [ ]:
!pip install evaluate

In [ ]:
!pip install bert_score

Now we need an evaluation metric. I choose bert_score for this example. Try to find a better one!

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("bertscore")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, lang = 'en')
    for k, v in result.items():
      if all(isinstance(item, (int, float)) for item in v):
        result[k] = round(np.mean(v) * 100, 4)
      else:
        print(f"Non-numeric values found for key {k}: {v}")
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

We can set here the hyperparameters

In [ ]:
from transformers import Seq2SeqTrainingArguments

To speed up the process we select `num_train_epochs = 0.01`

In [ ]:
training_args = Seq2SeqTrainingArguments(output_dir = "test_trainer", evaluation_strategy= "no",per_device_train_batch_size=4, per_device_eval_batch_size=1, eval_accumulation_steps = 1, num_train_epochs = 0.01,predict_with_generate=True)

Finally we create the Trainer object

In [ ]:
from transformers import Seq2SeqTrainer

In [ ]:
trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = small_eval_dataset,
    compute_metrics = compute_metrics,


)

In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=42, training_loss=2.0057373046875, metrics={'train_runtime': 17.8566, 'train_samples_per_second': 9.219, 'train_steps_per_second': 2.352, 'total_flos': 31229604790272.0, 'train_loss': 2.0057373046875, 'epoch': 0.01})

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Trainer is attempting to log a value of "roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.35.2)" of type <class 'str'> for key "eval/hashcode" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Non-numeric values found for key hashcode: roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.35.2)


{'eval_loss': 1.5317593812942505,
 'eval_precision': 87.5834,
 'eval_recall': 80.1574,
 'eval_f1': 83.7006,
 'eval_hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.35.2)',
 'eval_gen_len': 15.8,
 'eval_runtime': 15.8199,
 'eval_samples_per_second': 0.632,
 'eval_steps_per_second': 0.632,
 'epoch': 0.01}